In [2]:
cd ../..

/Users/hao/workspace/hpi_de/4th_Semester/Applied Machine Learning/sensor-data-gans


In [3]:
import numpy as np
import pandas as pd
import tensorflow
from sklearn.utils import shuffle
from utils.windowing import windowing_dataframe, transform_windows_df

## Read Data

In [4]:
train_df = pd.read_hdf('./datasets/mydata/train_df.h5')
val_df = pd.read_hdf('./datasets/mydata/val_df.h5')
test_df = pd.read_hdf('./datasets/mydata/test_df.h5')

## Windowing Timeseries Data

In [5]:
window_size = 5*50
step_size = int(window_size/2)
col_names = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z', 'userAcceleration.c']
method ='sliding'

In [6]:
train_windowed_df = windowing_dataframe(train_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)
val_windowed_df = windowing_dataframe(val_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)
test_windowed_df = windowing_dataframe(test_df, window_size=window_size, step_or_sample_size=step_size, col_names=col_names, method=method)

## Classifiers

### SVM

In [6]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

In [7]:
input_cols = ['userAcceleration.c']
x_train, y_train = transform_windows_df(train_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
x_val, y_val = transform_windows_df(val_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)
x_test, y_test = transform_windows_df(test_windowed_df, input_cols=input_cols, one_hot_encode=False, as_channel=False)

In [8]:
x_train = np.concatenate([x_train, x_val])
y_train = np.concatenate([y_train, y_val])

In [9]:
x_train = x_train.reshape((len(x_train), window_size))
x_test = x_test.reshape((len(x_test), window_size))

In [10]:
svm_clf = SVC()
svm_clf.fit(x_train, y_train)

SVC()

#### train performance

In [11]:
y_train_head = svm_clf.predict(x_train)

In [12]:
print(classification_report(y_train, y_train_head))

              precision    recall  f1-score   support

         0.0       0.62      1.00      0.77      1977
         1.0       0.87      0.46      0.61      2256
         2.0       1.00      0.82      0.90       829

    accuracy                           0.73      5062
   macro avg       0.83      0.76      0.76      5062
weighted avg       0.80      0.73      0.72      5062



#### test performance

In [13]:
y_test_head = svm_clf.predict(x_test)

In [14]:
print(classification_report(y_test, y_test_head))

              precision    recall  f1-score   support

         0.0       0.69      1.00      0.82       403
         1.0       0.78      0.35      0.49       387
         2.0       0.67      0.78      0.72       173

    accuracy                           0.70       963
   macro avg       0.71      0.71      0.67       963
weighted avg       0.72      0.70      0.67       963



### CNN

In [65]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, MaxPool1D, GlobalMaxPool1D, Flatten, Dropout
from sklearn.metrics import classification_report

In [66]:
input_cols = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']
x_train, y_train = transform_windows_df(train_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)
x_val, y_val = transform_windows_df(val_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)
x_test, y_test = transform_windows_df(test_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)

In [67]:
input_shape = x_train[0].shape

### build neural net

In [68]:
model = Sequential()
model.add(Conv1D(8, kernel_size=5, activation='relu', input_shape=input_shape))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_13 (Conv1D)           (None, 246, 8)            128       
_________________________________________________________________
flatten_8 (Flatten)          (None, 1968)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 5907      
Total params: 6,035
Trainable params: 6,035
Non-trainable params: 0
_________________________________________________________________


In [69]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [70]:
x, y = shuffle(x_train, y_train)
model.fit(x, y, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
126/126 [==============================] - 0s 3ms/step - loss: 0.7394 - accuracy: 0.6749 - val_loss: 0.4074 - val_accuracy: 0.9498
Epoch 2/10
126/126 [==============================] - 0s 2ms/step - loss: 0.1963 - accuracy: 0.9593 - val_loss: 0.0693 - val_accuracy: 0.9894
Epoch 3/10
126/126 [==============================] - 0s 2ms/step - loss: 0.0580 - accuracy: 0.9918 - val_loss: 0.0517 - val_accuracy: 0.9846
Epoch 4/10
126/126 [==============================] - 0s 2ms/step - loss: 0.0317 - accuracy: 0.9958 - val_loss: 0.0297 - val_accuracy: 0.9971
Epoch 5/10
126/126 [==============================] - 0s 2ms/step - loss: 0.0209 - accuracy: 0.9975 - val_loss: 0.0287 - val_accuracy: 0.9961
Epoch 6/10
126/126 [==============================] - 0s 2ms/step - loss: 0.0149 - accuracy: 0.9978 - val_loss: 0.0333 - val_accuracy: 0.9942
Epoch 7/10
126/126 [==============================] - 0s 2ms/step - loss: 0.0114 - accuracy: 0.9988 - val_loss: 0.0315 - val_accuracy: 0.9952
Epoch 

#### training performance

In [83]:
model.evaluate(x, y)

126/126 [==============================] - 0s 1ms/step - loss: 0.0054 - accuracy: 0.9990


[0.0053551411256194115, 0.999006450176239]

In [71]:
y_head = model.predict(x)

In [80]:
print(classification_report(y.argmax(axis=-1), y_head.argmax(axis=-1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1547
           1       1.00      1.00      1.00      1810
           2       1.00      1.00      1.00       669

    accuracy                           1.00      4026
   macro avg       1.00      1.00      1.00      4026
weighted avg       1.00      1.00      1.00      4026



#### test performance

In [84]:
model.evaluate(x_test, y_test)

31/31 [==============================] - 0s 1ms/step - loss: 0.0249 - accuracy: 0.9938


[0.024869784712791443, 0.9937694668769836]

In [81]:
y_head = model.predict(x_test)

In [82]:
print(classification_report(y_test.argmax(axis=-1), y_head.argmax(axis=-1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       403
           1       0.99      0.99      0.99       387
           2       0.98      0.99      0.99       173

    accuracy                           0.99       963
   macro avg       0.99      0.99      0.99       963
weighted avg       0.99      0.99      0.99       963



### LSTM

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Flatten, Dropout
from sklearn.metrics import classification_report

In [8]:
input_cols = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']
x_train, y_train = transform_windows_df(train_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)
x_val, y_val = transform_windows_df(val_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)
x_test, y_test = transform_windows_df(test_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)

In [9]:
input_shape = x_train[0].shape

In [20]:
model = Sequential()
model.add(LSTM(8, return_sequences=True, input_shape=input_shape))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 250, 8)            384       
_________________________________________________________________
flatten_3 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 6003      
Total params: 6,387
Trainable params: 6,387
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [22]:
x, y = shuffle(x_train, y_train)
model.fit(x, y, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
126/126 [==============================] - 8s 67ms/step - loss: 0.5413 - accuracy: 0.7958 - val_loss: 0.1456 - val_accuracy: 0.9710
Epoch 2/10
126/126 [==============================] - 8s 64ms/step - loss: 0.0647 - accuracy: 0.9824 - val_loss: 0.0514 - val_accuracy: 0.9903
Epoch 3/10
126/126 [==============================] - 8s 63ms/step - loss: 0.0354 - accuracy: 0.9908 - val_loss: 0.0479 - val_accuracy: 0.9932
Epoch 4/10
126/126 [==============================] - 8s 64ms/step - loss: 0.0287 - accuracy: 0.9928 - val_loss: 0.0468 - val_accuracy: 0.9932
Epoch 5/10
126/126 [==============================] - 8s 64ms/step - loss: 0.0190 - accuracy: 0.9953 - val_loss: 0.0486 - val_accuracy: 0.9952
Epoch 6/10
126/126 [==============================] - 8s 62ms/step - loss: 0.0210 - accuracy: 0.9945 - val_loss: 0.0385 - val_accuracy: 0.9971
Epoch 7/10
126/126 [==============================] - 8s 63ms/step - loss: 0.0128 - accuracy: 0.9963 - val_loss: 0.0336 - val_accuracy: 0.9971

#### training performance

In [23]:
model.evaluate(x, y)

126/126 [==============================] - 1s 10ms/step - loss: 0.0051 - accuracy: 0.9990


[0.005127791315317154, 0.999006450176239]

In [24]:
y_head = model.predict(x)

In [25]:
print(classification_report(y.argmax(axis=-1), y_head.argmax(axis=-1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1547
           1       1.00      1.00      1.00      1810
           2       1.00      1.00      1.00       669

    accuracy                           1.00      4026
   macro avg       1.00      1.00      1.00      4026
weighted avg       1.00      1.00      1.00      4026



#### test performance

In [26]:
model.evaluate(x_test, y_test)

31/31 [==============================] - 0s 10ms/step - loss: 0.0202 - accuracy: 0.9948


[0.020177213475108147, 0.9948078989982605]

In [27]:
y_head = model.predict(x_test)

In [28]:
print(classification_report(y_test.argmax(axis=-1), y_head.argmax(axis=-1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       403
           1       0.99      1.00      0.99       387
           2       1.00      0.98      0.99       173

    accuracy                           0.99       963
   macro avg       1.00      0.99      0.99       963
weighted avg       0.99      0.99      0.99       963



### BiLSTM

In [30]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Bidirectional, Dropout, Flatten, Dropout
from sklearn.metrics import classification_report

In [31]:
input_cols = ['userAcceleration.x', 'userAcceleration.y', 'userAcceleration.z']
x_train, y_train = transform_windows_df(train_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)
x_val, y_val = transform_windows_df(val_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)
x_test, y_test = transform_windows_df(test_windowed_df, input_cols=input_cols, one_hot_encode=True, as_channel=False)

In [32]:
input_shape = x_train[0].shape

In [37]:
model = Sequential()
model.add(Bidirectional(LSTM(8, return_sequences=True), input_shape=input_shape))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_4 (Bidirection (None, 250, 16)           768       
_________________________________________________________________
flatten_8 (Flatten)          (None, 4000)              0         
_________________________________________________________________
dense_9 (Dense)              (None, 3)                 12003     
Total params: 12,771
Trainable params: 12,771
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [39]:
x, y = shuffle(x_train, y_train)
model.fit(x, y, batch_size=32, epochs=10, validation_data=(x_val, y_val))

Epoch 1/10
126/126 [==============================] - 10s 77ms/step - loss: 0.3525 - accuracy: 0.8785 - val_loss: 0.0469 - val_accuracy: 0.9952
Epoch 2/10
126/126 [==============================] - 9s 73ms/step - loss: 0.0310 - accuracy: 0.9911 - val_loss: 0.0368 - val_accuracy: 0.9952
Epoch 3/10
126/126 [==============================] - 9s 74ms/step - loss: 0.0194 - accuracy: 0.9948 - val_loss: 0.0333 - val_accuracy: 0.9961
Epoch 4/10
126/126 [==============================] - 9s 75ms/step - loss: 0.0125 - accuracy: 0.9965 - val_loss: 0.0398 - val_accuracy: 0.9952
Epoch 5/10
126/126 [==============================] - 10s 76ms/step - loss: 0.0085 - accuracy: 0.9978 - val_loss: 0.0409 - val_accuracy: 0.9961
Epoch 6/10
126/126 [==============================] - 10s 77ms/step - loss: 0.0080 - accuracy: 0.9980 - val_loss: 0.0396 - val_accuracy: 0.9971
Epoch 7/10
126/126 [==============================] - 10s 77ms/step - loss: 0.0065 - accuracy: 0.9990 - val_loss: 0.0561 - val_accuracy: 0.

#### training performance

In [40]:
model.evaluate(x, y)

126/126 [==============================] - 2s 13ms/step - loss: 0.0062 - accuracy: 0.9990


[0.006229473743587732, 0.999006450176239]

In [41]:
y_head = model.predict(x)

In [42]:
print(classification_report(y.argmax(axis=-1), y_head.argmax(axis=-1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1547
           1       1.00      1.00      1.00      1810
           2       1.00      1.00      1.00       669

    accuracy                           1.00      4026
   macro avg       1.00      1.00      1.00      4026
weighted avg       1.00      1.00      1.00      4026



#### test performance

In [43]:
model.evaluate(x_test, y_test)

31/31 [==============================] - 0s 12ms/step - loss: 0.0412 - accuracy: 0.9875


[0.04120521619915962, 0.9875389337539673]

In [44]:
y_head = model.predict(x_test)

In [45]:
print(classification_report(y_test.argmax(axis=-1), y_head.argmax(axis=-1)))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       403
           1       0.99      0.97      0.98       387
           2       0.95      0.99      0.97       173

    accuracy                           0.99       963
   macro avg       0.98      0.99      0.98       963
weighted avg       0.99      0.99      0.99       963

